# `MEDUSA`
aka. Dynamic-Prospective LCA aka. Union(premise, temporalis)

In [1]:
from bw_temporalis import easy_timedelta_distribution, TemporalDistribution, easy_datetime_distribution
import sys
import os
sys.path.append(os.path.realpath('../'))
from medusa.edge_extractor import *
from medusa.matrix_modifier import *
from medusa.medusa_lca import *
from medusa.timeline_builder import *
import bw2data as bd
import bw2calc as bc
import numpy as np
import pandas as pd

In [2]:
from tests.databases import *
# db_abc_simple() # works
# db_abc_C_to_E() # works
# db_abc_C_and_B_to_E() # BREAKS THINGS
# db_abc_B_to_E() # BREAKS THINGS
db_abc_B_to_E_simplified() # BREAKS THINGS
# db_abc_B_to_E_simplified_and_E_with_TD()

100%|██████████| 1/1 [00:00<00:00, 6626.07it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 54120.05it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 42799.02it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 4/4 [00:00<00:00, 81442.80it/s]

Vacuuming database 


In [3]:
demand = {('foreground', 'A'): 1}
gwp = ('GWP', 'example')

# Static LCA

In [4]:
slca = bc.LCA(demand, gwp)
slca.lci()
slca.lcia()
print(f'Static LCA score: {slca.score}')

Static LCA score: 122.8499967455864


# `MEDUSA` LCA

In [5]:
SKIPPABLE = [node.id for node in bd.Database('background_2020')] + [
    node.id for node in bd.Database('background_2024')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

Now we can do the graph traversal and create a timeline of edges:

In [6]:
eelca = EdgeExtractor(slca, edge_filter_function=filter_function)
timeline = eelca.build_edge_timeline()

Starting graph traversal
Calculation count: 8
7 6


In [7]:
timeline[-2]

Edge(distribution=TemporalDistribution instance with 2 values and total: 32.5, leaf=True, consumer=8, producer=2, td_producer=TemporalDistribution instance with 1 values and total: 13, td_consumer=TemporalDistribution instance with 1 values and total: 5, abs_td_producer=TemporalDistribution instance with 2 values and total: 26, abs_td_consumer=TemporalDistribution instance with 2 values and total: 10)

In [8]:
database_date_dict = {
            datetime.strptime("2008", "%Y"): 'background_2008',
            datetime.strptime("2024", "%Y"): 'background_2024',
        }

timeline_df = create_grouped_edge_dataframe(timeline, database_date_dict, interpolation_type="linear")
timeline_df

All databases in database_date_dict exist as brightway project databases


,hash_producer,date_producer,producer,producer_name,hash_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,2013,2013-01-01,2,C,2018,2018-01-01,8,B,13.0,"{'background_2008': 0.6873716632443532, 'backg..."
1,2014,2014-01-01,2,C,2019,2019-01-01,8,B,13.0,"{'background_2008': 0.624914442162902, 'backgr..."
2,2015,2015-01-01,2,C,2020,2020-01-01,8,B,13.0,"{'background_2008': 0.5624572210814511, 'backg..."
3,2016,2016-01-01,2,C,2021,2021-01-01,8,B,26.0,"{'background_2008': 0.5, 'background_2024': 0.5}"
4,2017,2017-01-01,2,C,2022,2022-01-01,8,B,13.0,"{'background_2008': 0.43737166324435317, 'back..."
5,2018,2018-01-01,8,B,2022,2022-01-01,6,E,3.2,"{'background_2008': 0.37491444216290215, 'back..."
6,2019,2019-01-01,8,B,2023,2023-01-01,6,E,3.2,"{'background_2008': 0.312457221081451, 'backgr..."
7,2020,2020-01-01,8,B,2022,2022-01-01,6,E,4.8,"{'background_2008': 0.25, 'background_2024': 0..."
8,2021,2021-01-01,8,B,2022,2022-01-01,7,D,5.0,"{'background_2008': 0.18737166324435317, 'back..."
9,2021,2021-01-01,8,B,2023,2023-01-01,6,E,4.8,"{'background_2008': 0.18737166324435317, 'back..."


In [9]:
demand_timing_dict = create_demand_timing_dict(timeline_df, demand)
dp = create_datapackage_from_edge_timeline(timeline_df, database_date_dict, demand_timing_dict)

In [10]:
fu, data_objs, remapping = prepare_medusa_lca_inputs(demand=demand, demand_timing_dict=demand_timing_dict, method=gwp) 
lca = bc.LCA(fu, data_objs = data_objs + [dp], remapping_dicts=remapping)
lca.lci()
lca.lcia()

In [11]:
print('New MEDUSA LCA Score:', lca.score)
print('Old static LCA Score:', slca.score)

New MEDUSA LCA Score: 123.71881278316832
Old static LCA Score: 122.8499967455864


In [12]:
df = pd.DataFrame(lca.technosphere_matrix.toarray())
df.rename(lca.dicts.activity.reversed, inplace=True, axis=0)
df.rename(lca.dicts.activity.reversed, inplace=True, axis=1)
df


,2,3,4,5,6,7,8,9,2002013,2002014,...,6002022,6002023,7002022,7002023,8002018,8002019,8002020,8002021,8002022,9002024
2,1.0,0.0,0.0,0.0,0.0,0.0,-13.0,0.0,-0.312628,-0.375086,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.687372,-0.624914,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
5,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
6,0.0,0.0,0.0,0.0,1.0,-2.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
7,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.5,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
8,0.0,0.0,0.0,0.0,-8.0,-5.0,1.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2002013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,...,0.0,0.0,0.0,0.0,-13.0,0.0,0.0,0.0,0.0,0.00
2002014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,-13.0,0.0,0.0,0.0,0.00


In [13]:
rem = lca.remapping_dicts['activity']
rem

{1: ('temporalis-bio', 'CO2'),
 2: ('background_2024', 'C'),
 3: ('background_2024', 'electricity_wind'),
 4: ('background_2008', 'C'),
 5: ('background_2008', 'electricity_wind'),
 6: ('foreground', 'E'),
 7: ('foreground', 'D'),
 8: ('foreground', 'B'),
 9: ('foreground', 'A')}

In [18]:
dict(lca.dicts.activity)

{2: 0,
 3: 1,
 4: 2,
 5: 3,
 6: 4,
 7: 5,
 8: 6,
 9: 7,
 2001999: 8,
 2002016: 9,
 2002017: 10,
 2002027: 11,
 2002028: 12,
 2002032: 13,
 2002033: 14,
 2002034: 15,
 6002022: 16,
 6002023: 17,
 7002022: 18,
 7002023: 19,
 8002004: 20,
 8002021: 21,
 8002022: 22,
 8002039: 23,
 9002024: 24}

In [15]:
abs = timeline[1].abs_td_consumer

In [17]:
abs.date

array(['2024-01-30T18:45:57'], dtype='datetime64[s]')

In [9]:
td = TemporalDistribution(
                            date=np.array(
                                [
                                    0,
                                ],
                                dtype="timedelta64[Y]",
                            ),
                            amount=np.array([1]),
                        )

In [11]:
td.date

array([0], dtype='timedelta64[s]')

In [ ]:
def join_datetime_and_timedelta_distributions(
        td_producer: TemporalDistribution, td_consumer: TemporalDistribution
    ) -> TemporalDistribution:
        """TODO: Needs description"""
        # if an edge does not have a TD, then return the consumer_td so that the timeline continues
        if isinstance(td_consumer, TemporalDistribution) and isinstance(
            td_producer, Number
        ):
            return td_consumer
        # Else, if both consumer and producer have a td (absolute and relative, respectively) join to TDs
        if isinstance(td_producer, TemporalDistribution) and isinstance(
            td_consumer, TemporalDistribution
        ):
            if not (td_consumer.date.dtype == datetime_type):
                raise ValueError(
                    f"`td_consumer.date` must have dtype `datetime64[s]`, but got `{td_consumer.date.dtype}`"
                )
            if not (td_producer.date.dtype == timedelta_type):
                raise ValueError(
                    f"`td_producer.date` must have dtype `timedelta64[s]`, but got `{td_producer.date.dtype}`"
                )
            date = (
                td_consumer.date.reshape((-1, 1)) + td_producer.date.reshape((1, -1))
            ).ravel()
            amount = np.array(len(td_consumer) * [td_producer.amount]).ravel()
            return TemporalDistribution(date, amount)
        else:
            raise ValueError(
                "Can't join TemporalDistribution and something else: Trying with {} and {}".format(
                    type(td1), type(td2)
                )
            )